In [1]:
import numpy as np
import pickle
import pandas as pd
import json
import math
import torch
import string
from random import randint
from collections import defaultdict, Counter
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torchvision
import os
import torch.nn.functional as F
import torch.nn as nn
from torchvision import utils
from itertools import chain
import torch.utils.data as data_utils

import random
from tqdm import tqdm
from inception_score import get_inception_score

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
print(torch.__version__)


1.10.2


In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.conv1=nn.ConvTranspose2d(in_channels=100, out_channels=1024, kernel_size=4, stride=1, padding=0)
        self.batch1=nn.BatchNorm2d(num_features=1024)
        self.activ1=nn.ReLU(True)
        # State (1024x4x4)
        self.conv2=nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1)
        self.batch2=nn.BatchNorm2d(num_features=512)
        self.activ2=nn.ReLU(True)
        # State (512x8x8)
        self.conv3=nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1)
        self.batch3=nn.BatchNorm2d(num_features=256)
        self.activ3=nn.ReLU(True)
        # State (256x16x16)
        self.conv4=nn.ConvTranspose2d(in_channels=256, out_channels=3, kernel_size=4, stride=2, padding=1)
        self.output=nn.Tanh()
    def forward(self,x):
        x=self.conv1(x)
        x=self.batch1(x)
        x=self.activ1(x)
        x=self.conv2(x)
        x=self.batch2(x)
        x=self.activ2(x)     
        x=self.conv3(x)
        x=self.batch3(x)
        x=self.activ3(x)
        x=self.conv4(x)
        x=self.output(x)
        return x

In [5]:
# class Generator(torch.nn.Module):
#     def __init__(self, channels=3):
#         super().__init__()
#         # Filters [1024, 512, 256]
#         # Input_dim = 100
#         # Output_dim = C (number of channels)
#         self.main_module = nn.Sequential(
#             # Z latent vector 100
#             nn.ConvTranspose2d(in_channels=100, out_channels=1024, kernel_size=4, stride=1, padding=0),
#             nn.BatchNorm2d(num_features=1024),
#             nn.ReLU(True),

#             # State (1024x4x4)
#             nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(num_features=512),
#             nn.ReLU(True),

#             # State (512x8x8)
#             nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(num_features=256),
#             nn.ReLU(True),

#             # State (256x16x16)
#             nn.ConvTranspose2d(in_channels=256, out_channels=channels, kernel_size=4, stride=2, padding=1))
#             # output of main module --> Image (Cx32x32)

#         self.output = nn.Tanh()

#     def forward(self, x):
#         x = self.main_module(x)
#         return self.output(x)

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1=nn.Conv2d(in_channels=3, out_channels=256, kernel_size=4, stride=2, padding=1)
        self.activ1=nn.LeakyReLU(0.2, inplace=True)
        # State (1024x4x4)
        self.conv2=nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1)
        self.batch2=nn.BatchNorm2d(512)
        self.activ2=nn.LeakyReLU(0.2, inplace=True)
        self.conv3=nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1)
        self.batch3=nn.BatchNorm2d(1024)
        self.activ3=nn.LeakyReLU(0.2, inplace=True)
        # State (1024x4x4)

        self.conv4=nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0)
        self.output=nn.Sigmoid()
    def forward(self,x):
        x=self.conv1(x)
        x=self.activ1(x)
        x=self.conv2(x)
        x=self.batch2(x)
        x=self.activ2(x)     
        x=self.conv3(x)
        x=self.batch3(x)
        x=self.activ3(x)
        x=self.conv4(x)
        x=self.output(x)
        return x

In [7]:
# class Discriminator(torch.nn.Module):
#     def __init__(self, channels=3):
#         super().__init__()
#         # Filters [256, 512, 1024]
#         # Input_dim = channels (Cx64x64)
#         # Output_dim = 1
#         self.main_module = nn.Sequential(
#             # Image (Cx32x32)
#             nn.Conv2d(in_channels=channels, out_channels=256, kernel_size=4, stride=2, padding=1),
#             nn.LeakyReLU(0.2, inplace=True),

#             # State (256x16x16)
#             nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(512),
#             nn.LeakyReLU(0.2, inplace=True),

#             # State (512x8x8)
#             nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(1024),
#             nn.LeakyReLU(0.2, inplace=True))
#             # outptut of main module --> State (1024x4x4)

#         self.output = nn.Sequential(
#             nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0),
#             # Output 1
#             nn.Sigmoid())

#     def forward(self, x):
#         x = self.main_module(x)
#         return self.output(x)


In [8]:
disc_model=Discriminator()
gen_model=Generator()
disc_model.to(device)
gen_model.to(device)


Generator(
  (conv1): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1))
  (batch1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ1): ReLU(inplace=True)
  (conv2): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ2): ReLU(inplace=True)
  (conv3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ3): ReLU(inplace=True)
  (conv4): ConvTranspose2d(256, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (output): Tanh()
)

In [9]:
gen_model

Generator(
  (conv1): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1))
  (batch1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ1): ReLU(inplace=True)
  (conv2): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ2): ReLU(inplace=True)
  (conv3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ3): ReLU(inplace=True)
  (conv4): ConvTranspose2d(256, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (output): Tanh()
)

In [10]:
disc_model

Discriminator(
  (conv1): Conv2d(3, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (activ1): LeakyReLU(negative_slope=0.2, inplace=True)
  (conv2): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ2): LeakyReLU(negative_slope=0.2, inplace=True)
  (conv3): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (batch3): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ3): LeakyReLU(negative_slope=0.2, inplace=True)
  (conv4): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1))
  (output): Sigmoid()
)

In [11]:
def create_dataset(batchsize):
    trans = transforms.Compose([
            transforms.Resize(32),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])

    train_dataset = torchvision.datasets.CIFAR10(root='./files/', train=True, download=True, transform=trans)
    test_dataset = torchvision.datasets.CIFAR10(root='./files/', train=False, download=True, transform=trans)

    
    train_dataloader = data_utils.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = data_utils.DataLoader(test_dataset,  batch_size=batch_size, shuffle=True)
    return train_dataloader,test_dataloader

In [12]:
learn_rate=0.0002
epochs=50
batch_size=32
beta_1,beta_2=0.5,0.999

In [13]:
loss=nn.BCELoss()
gen_optim=torch.optim.Adam(gen_model.parameters(), lr=learn_rate,betas=(beta_1,beta_2))
disc_optim=torch.optim.Adam(disc_model.parameters(), lr=learn_rate,betas=(beta_1,beta_2))

In [14]:
def save_model(gen,disc):
    torch.save(gen.state_dict(), './generator_dcgan.pkl')
    torch.save(disc.state_dict(), './discriminator_dcgan.pkl')
    print('Models save to ./generator.pkl & ./discriminator.pkl ')

In [15]:
# def train(train_loader,loss):
#     for images,_ in train_loader:
        
#         images=images.to(device)
#         disc_model.to(device)
#         gen_model.to(device)
#         noise = torch.rand((images.shape[0], 100, 1, 1)).to(device)
#         real_labels = torch.ones(images.shape[0]).to(device)
#         fake_labels = torch.zeros(images.shape[0]).to(device)
#         # Train discriminator

#         # Compute BCE_Loss using real images
#         real_image_disc_output = disc_model(images)
#         d_loss_real = loss(real_image_disc_output.flatten(), real_labels)
#         real_score = real_image_disc_output

#         # Compute BCE Loss using fake images

#         fake_images_generator_output = gen_model(noise)
#         output_using_fake_image = disc_model(fake_images_generator_output)
#         d_loss_fake =loss(output_using_fake_image.flatten(), fake_labels)
#         fake_score = output_using_fake_image

#         # Optimize discriminator
#         d_loss = d_loss_real + d_loss_fake
#         disc_model.zero_grad()
#         d_loss.backward()
#         disc_optim.step()

#         # Train generator
#         # Compute loss with fake images
#         noise = torch.rand((images.shape[0], 100, 1, 1)).to(device)
#         fake_images_generator_output = gen_model(noise)
        
#         outputs_of_discrimiator = disc_model(fake_images_generator_output)
        
#         g_loss = loss(outputs_of_discrimiator.flatten(), real_labels)

#         # Optimize generator
#         disc_model.zero_grad()
#         gen_model.zero_grad()
#         g_loss.backward()
#         gen_optim.step()
    

        

In [16]:
 def train(train_loader):
        generator_iter = 0
        file = open("inception_score_graph.txt", "w")
        for epoch in tqdm(range(epochs)):

            for i, (images, _) in enumerate(train_loader):
                # Check if round number of batches
                if i == train_loader.dataset.__len__() // batch_size:
                    break

                z = Variable(torch.randn(batch_size, 100, 1, 1)).to(device)
                real_labels = torch.ones(batch_size).to(device)
                fake_labels = torch.zeros(batch_size).to(device)

                images=images.to(device)


                # Train discriminator
                # Compute BCE_Loss using real images
                outputs = disc_model(images)
                d_loss_real = loss(outputs.flatten(), real_labels)
                real_score = outputs

                # Compute BCE Loss using fake images
                z = Variable(torch.randn(batch_size, 100, 1, 1)).to(device)

                fake_images = gen_model(z)
                outputs = disc_model(fake_images)
                d_loss_fake = loss(outputs.flatten(), fake_labels)
                fake_score = outputs

                # Optimize discriminator
                d_loss = d_loss_real + d_loss_fake
                disc_model.zero_grad()
                d_loss.backward()
                disc_optim.step()

                # Train generator
                # Compute loss with fake images
                z = Variable(torch.randn(batch_size, 100, 1, 1)).to(device)

                fake_images = gen_model(z)
                outputs = disc_model(fake_images)
                g_loss = loss(outputs.flatten(), real_labels)

                # Optimize generator
                disc_model.zero_grad()
                gen_model.zero_grad()
                g_loss.backward()
                gen_optim.step()
                generator_iter += 1


                if generator_iter % 1000 == 0:
                    # Workaround because graphic card memory can't store more than 800+ examples in memory for generating image
                    # Therefore doing loop and generating 800 examples and stacking into list of samples to get 8000 generated images
                    # This way Inception score is more correct since there are different generated examples from every class of Inception model
                    sample_list = []
                    for i in range(10):
                        z = Variable(torch.randn(800, 100, 1, 1)).to(device)
                        samples = gen_model(z)
                        sample_list.append(samples.data.cpu().numpy())
                    
                    # Flattening list of lists into one list of numpy arrays
                    new_sample_list = list(chain.from_iterable(sample_list))
                    print("Calculating Inception Score over 8k generated images")
                    # Feeding list of numpy arrays
                    inception_score = get_inception_score(new_sample_list, cuda=True, batch_size=32,
                                                          resize=True, splits=10)
                    print('Epoch-{}'.format(epoch + 1))

                    if not os.path.exists('training_result_images/'):
                        os.makedirs('training_result_images/')

                    # Denormalize images and save them in grid 8x8
                    z = Variable(torch.randn(800, 100, 1, 1)).to(device)
                    samples = gen_model(z)
                    samples = samples.mul(0.5).add(0.5)
                    samples = samples.data.cpu()[:64]
                    grid = utils.make_grid(samples)
                    utils.save_image(grid, 'training_result_images/img_generatori_iter_{}.png'.format(str(generator_iter).zfill(3)))

                    #print("Inception score: {}".format(inception_score))

                    # Write to file inception_score, gen_iters, time
                    output = str(generator_iter) + " " + str(inception_score[0]) + "\n"
                    file.write(output)



        file.close()

        # Save the trained parameters
        save_model(gen_model,disc_model)


In [17]:
def generate_img(noise, number_of_images):
    samples = gen_model(noise).data.cpu().numpy()[:number_of_images]
    generated_images = []
    for sample in samples:
        generated_images.append(sample.reshape(3, 32, 32))
    return generated_images

In [18]:
def load_model(G_model_filename,D_model_filename):
    D_model_path = os.path.join(os.getcwd(), D_model_filename)
    G_model_path = os.path.join(os.getcwd(), G_model_filename)
    disc_model.load_state_dict(torch.load(D_model_path))
    gen_model.load_state_dict(torch.load(G_model_path))
    print('Generator model loaded from {}.'.format(G_model_path))
    print('Discriminator model loaded from {}-'.format(D_model_path))

In [19]:
epochs

50

In [20]:
train_loader,_=create_dataset(batch_size)
train(train_loader)
save_model(gen_model,disc_model)


Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/50 [00:00<?, ?it/s]

Calculating Inception Score over 8k generated images


/home/nkulshr/.conda/envs/DL/lib/python3.9/site-packages/torch/nn/functional.py:3631: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/nkulshr/DL_HW3/inception_score.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x).data.cpu().numpy()


Epoch-1


...

Calculating Inception Score over 8k generated images
Epoch-49
Calculating Inception Score over 8k generated images
Epoch-49


 98%|█████████▊| 49/50 [2:05:48<02:34, 154.48s/it]

Calculating Inception Score over 8k generated images
Epoch-50
Calculating Inception Score over 8k generated images
Epoch-50


100%|██████████| 50/50 [2:08:28<00:00, 154.16s/it]


Models save to ./generator.pkl & ./discriminator.pkl 
Models save to ./generator.pkl & ./discriminator.pkl 


In [21]:
# load_model('./generator_dcgan.pkl','./discriminator_dcgan.pkl')

In [22]:
noise = torch.rand((64, 100, 1, 1)).to(device)
number_of_image=10
# generate_img(noise,number_of_image)

In [23]:
def evaluate(test_loader, D_model_path='./discriminator_dcgan.pkl', G_model_path='./generator_dcgan.pkl'):
#     load_model( G_model_path,D_model_path)
    z = torch.randn(batch_size, 100, 1, 1).to(device)
    samples = gen_model(z)
    samples = samples.mul(0.5).add(0.5)
    samples = samples.data.cpu()
    grid = utils.make_grid(samples)
    print("Grid of 8x8 images saved to 'dgan_model_image.png'.")
    utils.save_image(grid, 'dgan_model_image.png')

In [24]:
_,test_loader=create_dataset(batch_size)
evaluate(test_loader)

Files already downloaded and verified
Files already downloaded and verified
Grid of 8x8 images saved to 'dgan_model_image.png'.
